# Get map locations

## Read dataframe from csv

In [1]:
import pandas as pd

df1 = pd.read_csv("export_files\gas_prices_20220415_160024.csv")
df1.set_index("place_id",inplace=True)
df1.dropna(subset=["latitude","longitude"],inplace=True)

df2 = pd.read_csv("export_files\gas_address_20220415_160024.csv")
df2.set_index("place_id",inplace=True)

df = df1.merge(df2,how="left",left_index=True,right_index=True)
df

,name,cree_id,latitude,longitude,regular,premium,diesel,amenity,road,hamlet,city,county,state,ISO3166-2-lvl4,postcode
place_id,,,,,,,,,,,,,,,
2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.92140,22.49,24.99,21.99,Pemex,Boulevard de los Insurgentes,El Porvenir,Tijuana,Municipio de Tijuana,Baja California,MX-BCN,22223
2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.30370,-99.74484,21.25,23.19,23.09,NaN,MEX 45,NaN,NaN,Nopala de Villagrán,Hidalgo,MX-HID,NaN
2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.45140,17.28,19.20,NaN,Pemex E02526,Avenida Licenciado Adolfo López Mateos,NaN,Ciudad Juárez,Municipio de Juárez,Chihuahua,MX-CHH,94158
2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.30420,21.79,24.39,23.54,Pemex,Anillo Periférico Norte Manuel Gómez Morín,NaN,Guadalajara,NaN,Jalisco,MX-JAL,44390
2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,17.79,19.99,NaN,NaN,NaN,NaN,Reynosa,Municipio de Reynosa,Tamaulipas,MX-TAM,88500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27758,SUMINISTRO DE COMBUSTIBLES EL SEIS S. DE R.L. ...,PL/23961/EXP/ES/2022,20.88375,-102.70250,NaN,NaN,NaN,NaN,Carretera Tepatitlán-Las Adjuntas,NaN,NaN,San Ignacio Cerro Gordo,Jalisco,MX-JAL,47601
27759,GASOLINERA SACSALUM SA DE CV,PL/23960/EXP/ES/2022,16.32370,-91.97680,NaN,NaN,NaN,NaN,Chiapas 218,NaN,NaN,Las Margaritas,Chiapas,MX-CHP,30187
27764,"AEROPUERTO INTERNACIONAL FELIPE ANGELES, SOCIE...",PL/23974/EXP/ES/2022,19.72579,-99.03263,NaN,NaN,NaN,NaN,Acceso a Aeropuerto Int. Felipe Angeles,NaN,NaN,Nextlalpan,Estado de México,MX-MEX,55789


## Fill na Prices Values

In [7]:
df["regular"].fillna(df["regular"].mean(), inplace=True)
df["premium"].fillna(df["premium"].mean(), inplace=True)
df["diesel"].fillna(df["diesel"].mean(), inplace=True)
df[["regular","premium","diesel"]]

,regular,premium,diesel
place_id,,,
2039,22.490000,24.990000,21.990000
2040,21.250000,23.190000,23.090000
2041,17.280000,19.200000,22.937578
2042,21.790000,24.390000,23.540000
2043,17.790000,19.990000,22.937578
...,...,...,...
27758,21.430554,23.371421,22.937578
27759,21.430554,23.371421,22.937578
27764,21.430554,23.371421,22.937578


## Drop not valid locations

In [26]:
df[["state", "ISO3166-2-lvl4"]][df["state"].isnull()]
non_state = df[["state", "ISO3166-2-lvl4"]][df["state"].isnull()].index
df.drop(non_state, axis=0, inplace=True)

,state,ISO3166-2-lvl4
place_id,,


## Export AIO File

In [28]:
df.to_csv("export_files\gas_stations_aio.csv")

## Getting 150 nearest gas stations function 3

In [2]:
def distances(latitude: float, longitude: float):
    import geopy.distance
    distances = []
    for idx in df.index:
        distances.append(
            geopy.distance.geodesic(
                (latitude, longitude),
                (df.loc[idx,"latitude"], df.loc[idx,"longitude"])
                ).km
            )
    distances = pd.DataFrame(distances, columns=["distances"])
    distances["place_id"] = df.index
    distances.set_index("place_id",inplace=True)
    # Find gas stations 5 km around from position
    distances_idx = distances["distances"][distances["distances"] < 5].index
    return list(distances_idx)

## Popup text info

In [3]:
idx = 2043
def popup_info(idx):
      return """<b>PLACE INFO</b>
            <br>Name: {}
            <br>Location: {}, {}
            <br>County: {}
            <br>State: {}
            <br>Postcode: {}
            <br><br><b>PRICES</b>
            <br>Regular: ${}, Premium ${}, Diesel ${}""".format(df.loc[idx, "name"],
                                                              df.loc[idx, "road"],
                                                              df.loc[idx, "hamlet"],
                                                              df.loc[idx, "county"],
                                                              df.loc[idx, "state"],
                                                              df.loc[idx, "postcode"],
                                                              df.loc[idx, "regular"],
                                                              df.loc[idx, "premium"],
                                                              df.loc[idx, "diesel"])
print(popup_info(2043))

<b>PLACE INFO</b>
            <br>Name: PETROMAX, S.A. DE C.V.
            <br>Location: nan, nan
            <br>County: Municipio de Reynosa
            <br>State: Tamaulipas
            <br>Postcode: 88500
            <br><br><b>PRICES</b>
            <br>Regular: $17.79, Premium $19.99, Diesel $nan


## Plot map with locations

In [4]:
import folium

# Select location
center = [25.69651, -100.31676]

# gas type ["regular","premium","diesel"]
gas_type = "regular"

map = folium.Map(
    location = [center[0],center[1]],
    zoom_start = 13
    )

distance_index = distances(center[0],center[1])

max_price = df.loc[distance_index, gas_type].max()
min_price = df.loc[distance_index, gas_type].min()

thresold1 = df.loc[distance_index,gas_type].quantile(0.25)
thresold2 = df.loc[distance_index,gas_type].quantile(0.75)
#thresold1 = (max_price-min_price)*0.25 + min_price
#thresold2 = (max_price-min_price)*0.75 + min_price

for idx in distance_index:
    price = df.loc[idx,gas_type]
    
    if price < thresold1 and price > min_price:
        set_color = "lightgreen"
        
    elif price >= thresold1 and price <= thresold2:
        set_color = "blue"
        
    elif price > thresold2 and price < max_price:
        set_color = "lightred"
        
    if price == min_price:
        set_color = "green"
        
    elif price == max_price:
        set_color = "red"

    folium.Marker(
        location=[df.loc[idx,"latitude"], df.loc[idx,"longitude"]],
        popup=folium.Popup(popup_info(idx),min_width=400,max_width=400),
        icon=folium.Icon(color=set_color)
    ).add_to(map)

map

### Export map

In [5]:
from datetime import datetime
import re

#Creating name of file based in date
date_ex = re.sub("[-]", "", str(datetime.today()))
date_ex = re.sub("[ ]", "_", date_ex)
date_ex = re.sub("[:]", "", date_ex)
date_ex = date_ex[:-7]

map.save("export_files\map_{}.html".format(date_ex))

## Plot Choropleth
To plot choropleth map we need geojson file and we get that from [https://github.com/angelnmara](https://github.com/angelnmara/geojson/blob/master/mexicoHigh.json) repository.

### Dataframe Average prices

In [5]:
mean_prices = df.groupby(by=["state", "ISO3166-2-lvl4"]).mean()[
    ["regular"]].sort_values(by="regular")
mean_prices.reset_index(inplace=True)
mean_prices

,state,ISO3166-2-lvl4,regular
0,Tamaulipas,MX-TAM,19.157974
1,Chihuahua,MX-CHH,20.220768
2,Tabasco,MX-TAB,20.649217
3,Tlaxcala,MX-TLA,20.866905
4,Hidalgo,MX-HID,20.902292
5,Puebla,MX-PUE,20.989756
6,Morelos,MX-MOR,21.081243
7,Veracruz,MX-VER,21.091502
8,Baja California,MX-BCN,21.120660
9,Querétaro,MX-QUE,21.159040


### Creating Choropleth Map

In [7]:
map = folium.Map(location=[22.88222,-102.19593],zoom_start=6)

folium.Choropleth(
    geo_data="https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json",
    name="chlorophet",
    data=mean_prices,
    columns=["ISO3166-2-lvl4","regular"],
    key_on="feature.id",
    fill_color="OrRd",
    legend_name="AVG Gas price [Regular]"
).add_to(map)
map

### Export map

In [8]:
map.save("export_files\map_states_{}.html".format(date_ex))

### Choropleth Map with plotly

In [77]:
import plotly.express as px

fig = px.choropleth(
    mean_prices,
    geojson="https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json",
    locations="ISO3166-2-lvl4",
    hover_name="state",
    color="regular",
    color_continuous_scale="Viridis",
    scope="north america",
    labels={"regular": "Regular", "ISO3166-2-lvl4":"State code"},
    title="Average Gas prices by state",
    center={"lat":25,"lon":-99},
    width=800,
    height=500)

fig.update_xaxes(mirror=True)

fig.update_layout(margin={"r": 10, "t": 40, "l": 10, "b": 10})

fig.update_geos(showcountries=True, showcoastlines=True,
                showland=True, fitbounds="locations", bgcolor="lightblue")
fig.show()